In [0]:
%cd
  
!git clone --quiet https://github.com/tensorflow/models.git

!apt-get install -qq protobuf-compiler python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib PyDrive

!pip install -q pycocotools

%cd ~/models/research
!protoc object_detection/protos/*.proto --python_out=.


/root
Selecting previously unselected package libprotobuf10:amd64.
(Reading database ... 22280 files and directories currently installed.)
Preparing to unpack .../libprotobuf10_3.0.0-9.1ubuntu1_amd64.deb ...
Unpacking libprotobuf10:amd64 (3.0.0-9.1ubuntu1) ...
Selecting previously unselected package libprotoc10:amd64.
Preparing to unpack .../libprotoc10_3.0.0-9.1ubuntu1_amd64.deb ...
Unpacking libprotoc10:amd64 (3.0.0-9.1ubuntu1) ...
Selecting previously unselected package protobuf-compiler.
Preparing to unpack .../protobuf-compiler_3.0.0-9.1ubuntu1_amd64.deb ...
Unpacking protobuf-compiler (3.0.0-9.1ubuntu1) ...
Setting up libprotobuf10:amd64 (3.0.0-9.1ubuntu1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
Setting up libprotoc10:amd64 (3.0.0-9.1ubuntu1) ...
Setting up protobuf-compiler (3.0.0-9.1ubuntu1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
/root/models/research


In [0]:
import os
os.environ['PYTHONPATH'] += ':/root/models/research/:/root/models/research/slim/'

In [0]:
!python object_detection/builders/model_builder_test.py

In [0]:
%cd ..

/


In [0]:
!mkdir datos
%cd datos

/datos


In [0]:
!pip install PyDrive
import os
from zipfile import ZipFile
from shutil import copy
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
fileIdLabelmap = '1ARLwWBEZMO8MJsDhaFOxEs39-a8A_CTF'
fileIdTFtrain = '1md_suRPoUhiqaoG3DqXxRYx7MryjQMin'
fileIdTFtest = '16pRjhNmGbJPFVnvtjVbWxR5i5Q8IUk5a'

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#label map
fileName1 = fileIdLabelmap + '.pbtxt'
downloaded1 = drive.CreateFile({'id': fileIdLabelmap})
downloaded1.GetContentFile("label_map.pbtxt")
#tfrecord tes
fileName2 = fileIdTFtrain + '.record'
downloaded2 = drive.CreateFile({'id': fileIdTFtrain})
downloaded2.GetContentFile("train.record")
#tf train
fileName3 = fileIdTFtest + '.record'
downloaded3 = drive.CreateFile({'id': fileIdTFtest})
downloaded3.GetContentFile("test.record")

In [0]:
import os
import shutil
import glob
import urllib.request
import tarfile

MODEL = 'ssd_mobilenet_v2_coco_2018_03_29'
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = 'pretrained_model'

if not (os.path.exists(MODEL_FILE)):
  opener = urllib.request.URLopener()
  opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
  shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

In [0]:
!ls

label_map.pbtxt  pretrained_model  test.record	train.record


In [0]:
%cd datos

/datos


In [0]:
%mkdir training

In [0]:
%cd training/

/datos/training


In [0]:
!ls

In [0]:
from google.colab import files
uploaded = files.upload() 

Saving ssd_mobilenet_v2_coco.config to ssd_mobilenet_v2_coco.config


In [0]:
%cd datos

In [0]:
!ls

ssd_mobilenet_v2_coco.config


In [0]:
%cd ..

/


In [0]:
uploaded = files.upload() 

Saving train.py to train.py


In [0]:
!python /datos/train.py --logtostderr --train_dir=/datos/training/ --pipeline_config_path=/datos/training/ssd_mobilenet_v2_coco.config

Instructions for updating:
Use object_detection/model_main.py.
W1104 17:54:43.290825 140208661546880 tf_logging.py:125] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/platform/app.py:125: main (from __main__) is deprecated and will be removed in a future version.
Instructions for updating:
Use object_detection/model_main.py.
Instructions for updating:
Please switch to tf.train.create_global_step
W1104 17:54:43.392323 140208661546880 tf_logging.py:125] From /root/models/research/object_detection/legacy/trainer.py:265: create_global_step (from tensorflow.contrib.framework.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Please switch to tf.train.create_global_step
W1104 17:54:43.424312 140208661546880 tf_logging.py:125] num_readers has been reduced to 1 to match input file shards.
Instructions for updating:
Use `tf.data.experimental.parallel_interleave(...)`.
W1104 17:54:43.429005 140208661546880 tf_logging.py:125] Fro

In [0]:
!ls

In [0]:
!python root/models/research/object_detection/model_main.py \
    --pipeline_config_path=/datos/training/Faster.config\
    --model_dir=/datos \
    --alsologtostderr \
    --num_train_steps=50000 \
    --num_eval_steps=200

In [0]:
python model_main.py --pipeline_config_path=training/faster5.config --model_dir=training --alsologtostderr --num_train_steps=50000 --num_eval_steps=200